In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import curve_fit
from pandas import DataFrame as df
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import get_decimal_separator
from Data_setup_CHEM import data_set
import os
from galvani import BioLogic
from math import floor, log10
from datetime import datetime

fig_title = 'Title'
today_str = datetime.today().strftime('%Y-%m-%d')
fig = go.Figure()

# Autoscale axes
fig.update_xaxes(
    autorange=True,
    nticks=20,
    ticks='outside',
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    zeroline=False,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)
fig.update_yaxes(
    autorange=True,
    nticks=10,
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    ticks='outside',
    zeroline=True,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

colors = px.colors.qualitative.Dark24  # Use Dark24 color palette

fig.update_layout(
    xaxis_title="Cell Voltage (V)",
    yaxis_title="Current Density (mA/cm<sup>2</sup>)",
    margin={'t': 0, 'l': 0, 'b': 0, 'r': 0},
    legend_font_size=22,
    font=dict(
        family="Times New Roman, monospace",
        size=22,
        color="dimgrey",
    )
)

# Set default background (transparent paper with white display for JPEG later)
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 255)"
})

# Loop over each dataset (assumes files exist)
for i in data_set:
    file = data_set[i]['data_CA']
    # Read the file
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(
            file,
            encoding='ansi',
            sep="\t",
            decimal=get_decimal_separator(file),
            skiprows=range(headerlength)
        )
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
    
    # Automatically select the correct current column
    if '<I>/mA' in data.columns:
        current_column = '<I>/mA'
    else:
        current_column = 'I/mA'
    
    # Define variables for polarization data
    exp = data['Ns']                     # experiment/cycle numbers
    Nexp = int(exp.iloc[-1]) + 1           # total number of experiments (cycles)
    voltage = data['Ewe/V']
    current = data[current_column] / 5     # normalize current (divide by 5)
    
    Polarization = []
    for j in range(Nexp):
        # Compute the average voltage and current (and current standard deviation) for each experiment
        v = df.mean(voltage[exp == j])
        J = df.mean(current[exp == j])
        Jerr = df.std(current[exp == j])
        Polarization.append([v, J, Jerr])
    Polarization = np.array(Polarization)
    
    # Compute a linear fit on the polarization data to extract the x-intercept (voltage when current = 0)
    # (Assuming the first couple of experiments represent the high-frequency, low-current regime)
    if Polarization.shape[0] >= 2:
        # Fit: current = m * voltage + b
        coeffs = np.polyfit(Polarization[:,0], Polarization[:,1], 1)
        m, b = coeffs
        Rsubu = -b/m if m != 0 else None
        print(f"{data_set[i]['label']} Rsubu (x-intercept): {Rsubu}")
    
    fig.add_trace(go.Scatter(
        x=Polarization[:,0],
        y=Polarization[:,1],
        error_y=dict(
            type='data',
            array=Polarization[:,2],
            visible=True
        ),
        mode='markers',
        name=data_set[i]['label'],
        line_color=colors[data_set[i]['color_index']],
    ))

# Add a reference cell JV curve from CSV
ref_df = pd.read_csv('AEMEL/reference_cell_DioxideMaterialsE0621_5cm2_JVcurve_digitized.csv')
Reference = ref_df[['Voltage (V)', 'Current (A)']].values
Reference[:,1] *= 1000/5  # normalize the current (convert A to mA and divide by 5)
print(Reference)
fig.add_trace(go.Scatter(
    x=Reference[:,0],
    y=Reference[:,1],
    mode='markers',
    name="Reference",
    line_color=colors[0]
))

fig.update_layout(
    width=750,
    height=500,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0,
        font=dict(
            family="Times New Roman, monospace",
            size=22,
            color="dimgrey",
        )
    )
)

# First, save the SVG with a transparent background
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(0, 0, 0, 0)"
})
fig.write_image(f"{today_str} {fig_title} Chronopotentiometry.svg")

# Then update the background to white for display and for saving the JPEG
fig.update_layout({
    "plot_bgcolor": "rgba(255, 255, 255, 1)",
    "paper_bgcolor": "rgba(255, 255, 255, 1)"
})
fig.show()

fig.write_image(f"{today_str} {fig_title} Chronopotentiometry.jpeg")
